## Preprocessing

In [ ]:
from utils import reset_folder

reset_folder('./detected_faces')
print("Contents of detected_faces cleared.")

In [ ]:
from services.preprocessing import Preprocessing

image_paths = Preprocessing.get_image_paths('./data1')
Preprocessing.detect_and_save(image_paths, './detected_faces')

In [ ]:
from utils import count_file_and_folder

folder_count, total_file_count = count_file_and_folder('./detected_faces')
print(f"\nNumber of folders: {folder_count}")
print(f"Total number of files: {total_file_count}")

## Using VGG16 model

In [ ]:
import os
import shutil

def create_train_data(folder_path: str):
    os.mkdir('./resources/train')
    os.mkdir('./resources/val')
    for folder_name in os.listdir(folder_path):
        names = folder_name[5:].lower().split(' ')
        name = '_'.join(names)
        sub_folder_path = os.path.join(folder_path, folder_name)
        print('sub_folder_path', sub_folder_path)
        _, _, files = next(os.walk(sub_folder_path))
        _len = len(files)
        val_len = (_len - 2) * 0.2
        count = 0
        os.mkdir(f'./resources/train/{name}')
        os.mkdir(f'./resources/val/{name}')
        for image_name in os.listdir(sub_folder_path):
            src_path = os.path.join(sub_folder_path, image_name)
            pre_name = 'train'
            if count < val_len:
                pre_name = 'val'
            des_path = os.path.join(f'./resources/{pre_name}/{name}', f'{name}{count}.jpg')
            shutil.copy(src_path, des_path)
            count += 1

create_train_data('resources/train_detected_faces')

In [ ]:
count = 0

for actor_name in os.listdir('./resources/train'):
  count += 1
  sub_train_path = os.path.join('./resources/train', actor_name)
  sub_val_path = os.path.join('./resources/val', actor_name)
  
  _, _, train_files = next(os.walk(sub_train_path))
  _, _, val_files = next(os.walk(sub_val_path))

  print(actor_name, len(val_files) / (len(val_files) + len(train_files)))

print(count)


In [ ]:
from services.vgg16_model import get_dataset, get_vgg16_model, execute_model

train_data, test_data = get_dataset('./resources/train', './resources/val')
num_classes = train_data.num_classes
model = get_vgg16_model(num_classes)
execute_model(model, train_data, test_data, 100)